In [6]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import os

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

data_file = 'landmarks.csv'
header = ['shoulder_x', 'shoulder_y', 'elbow_x', 'elbow_y', 'wrist_x', 'wrist_y', 'label']

# Create CSV file if it doesn't exist
if not os.path.exists(data_file):
    with open(data_file, mode='w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)

cap = cv2.VideoCapture(0)
label = 0  # Start with illegal (0)
print("Collecting data for Illegal action. Press 'n' to switch to Legal.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        
        shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
        wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

        with open(data_file, mode='a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([shoulder[0], shoulder[1], elbow[0], elbow[1], wrist[0], wrist[1], label])
        
        # Draw landmarks and connections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=3),
                                  mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2))
    
    cv2.imshow('Data Collection', image)
    key = cv2.waitKey(10) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('n'):
        label = 1 if label == 0 else 0  # Toggle between 0 (Illegal) and 1 (Legal)
        print(f"Switched to {'Legal' if label == 1 else 'Illegal'} action collection.")

cap.release()
cv2.destroyAllWindows()

Switched to Legal action collection.
